In [1]:
import Pkg
if "MKL" in keys(Pkg.project().dependencies)
  using MKL
end
Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/Dev/Workspaces/ParallelLeastSquares/benchmark`


Status `~/Documents/Dev/Workspaces/ParallelLeastSquares/benchmark/Project.toml`
  [6e4b80f9] BenchmarkTools v1.6.0
  [8e7c35d0] BlockArrays v1.6.3
  [0a1fb500] BlockDiagonals v0.2.0
  [a93c6f00] DataFrames v1.7.0
  [31c24e10] Distributions v0.25.120
  [42fd0dbc] IterativeSolvers v0.9.4
  [9df87fff] MMDeweighting v1.0.0-DEV `https://github.com/qhengncsu/MMDeweighting.jl#main`
  [08abe8d2] PrettyTables v2.4.0
  [5d646e68] QBSolvers v0.1.0 `..`
  [6f49c342] RCall v0.14.8
  [37e2e46d] LinearAlgebra
  [10745b16] Statistics v1.10.0


In [2]:
versioninfo()

Julia Version 1.10.9
Commit 5595d20a287 (2025-03-10 12:51 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 20 × Intel(R) Core(TM) i9-10900KF CPU @ 3.70GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, skylake)
Threads: 1 default, 0 interactive, 1 GC (on 20 virtual cores)


In [3]:
using QBSolvers, LinearAlgebra, Distributions, Statistics

# OLS

In [4]:
n, p = 16*512, 2*512;
Σ = [(0.8)^abs(i-j) for i in 1:p, j in 1:p];
A = Transpose(rand(MvNormal(zeros(p), Σ), n)) |> Matrix;
x0 = 0.1*ones(p);
b = A*x0 + 1/sqrt(p)*randn(n);
x_init = zeros(p)
λ = 1e3
n_blk = p

1024

## QUB

In [5]:
x, r, stats = @time solve_OLS(A, b, x_init, p; maxiter=10^2, use_qub=true, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS(A, b, x_init, p; maxiter=10^4, use_qub=true, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            529ms /   8.0%           9.53MiB /  86.7%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   32.5ms   76.5%  32.5ms   8.00MiB   96.9%  8.00MiB
Diag QUB; normaliz...      1   10.0ms   23.5%  10.0ms    261KiB    3.1%   261KiB
  OLS loop                 1   6.25ms   14.7%  6.25ms    109KiB    1.3%   109KiB
    compute gₙ₊₁          99   2.37ms    5.6%  23.9μs   1.47KiB    0.0%    15.2B
      w = (AᵀA+λI)...     99   2.18ms    5.1%  22.0μs     0.00B    0.0%    0.00B
      axpy!               99    163μs    0.4%  1.65μs     0.00B    0.0%    0.00B
    init workspace         

  8.569588 seconds (12.51 M allocations: 810.639 MiB, 3.48% gc time, 99.54% compilation time)


────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           54.2ms /  99.8%           8.16MiB /  99.8%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   34.6ms   64.0%  34.6ms   8.00MiB   98.2%  8.00MiB
Diag QUB; normaliz...      1   19.5ms   36.0%  19.5ms    152KiB    1.8%   152KiB
  OLS loop                 1   19.3ms   35.7%  19.3ms    107KiB    1.3%   107KiB
    compute gₙ₊₁         592   13.9ms   25.7%  23.5μs   1.47KiB    0.0%    2.54B
      w = (AᵀA+λI)...    592   12.8ms   23.6%  21.6μs     0.00B    0.0%    0.00B
      axpy!              592    983μs    1.8%  1.66μs     0.00B    0.0%    0.00B
    init workspace         

  0.054529 seconds (1.36 k allocations: 8.257 MiB)


## L-BFGS, no preconditioner

In [6]:
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init; maxiter=10^2, precond=:none, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init; maxiter=10^4, precond=:none, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           38.1ms /  99.9%           8.27MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   31.6ms   83.0%  31.6ms   8.00MiB   96.7%  8.00MiB
OLS loop                   1   6.48ms   17.0%  6.48ms    276KiB    3.3%   276KiB
  w = (AᵀA+λI)dₙ₊₁       100   2.21ms    5.8%  22.1μs     0.00B    0.0%    0.00B
  init gradient            1   1.81ms    4.7%  1.81ms     0.00B    0.0%    0.00B
  compute residual         1   1.66ms    4.4%  1.66ms   64.0KiB    0.8%  64.0KiB
  compute search d...    100    501μs    1.3%  5.01μs     0.00B    0.0%    0.00B
  update L-BFGS cache    10

  1.357295 seconds (1.16 M allocations: 80.765 MiB, 1.63% gc time, 97.17% compilation time)


────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           49.2ms /  99.8%           8.28MiB /  99.8%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   33.7ms   68.7%  33.7ms   8.00MiB   96.7%  8.00MiB
OLS loop                   1   15.4ms   31.3%  15.4ms    276KiB    3.3%   276KiB
  w = (AᵀA+λI)dₙ₊₁       393   8.55ms   17.4%  21.8μs     0.00B    0.0%    0.00B
  compute search d...    393   2.13ms    4.3%  5.41μs     0.00B    0.0%    0.00B
  init gradient            1   1.83ms    3.7%  1.83ms     0.00B    0.0%    0.00B
  compute residual         1   1.73ms    3.5%  1.73ms   64.0KiB    0.8%  64.0KiB
  update L-BFGS cache    39

  0.049436 seconds (1.13 k allocations: 8.364 MiB)


In [7]:
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init; maxiter=10^2, precond=:qub, gtol=1e-4, lambda=λ, gram=true);
x, r, stats = @time solve_OLS_lbfgs(A, b, x_init; maxiter=10^4, precond=:qub, gtol=1e-4, lambda=λ, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           42.2ms /  99.9%           8.32MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   34.7ms   82.3%  34.7ms   8.00MiB   96.2%  8.00MiB
Diag QUB; normaliz...      1   7.47ms   17.7%  7.47ms    321KiB    3.8%   321KiB
  OLS loop                 1   7.26ms   17.2%  7.26ms    276KiB    3.2%   276KiB
    w = (AᵀA+λI)dₙ₊₁      91   2.12ms    5.0%  23.3μs     0.00B    0.0%    0.00B
    init gradient          1   2.03ms    4.8%  2.03ms     0.00B    0.0%    0.00B
    compute residual       1   1.90ms    4.5%  1.90ms   64.0KiB    0.8%  64.0KiB
    compute search...     9

  0.042681 seconds (1.23 k allocations: 8.411 MiB)


────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:           39.3ms /  99.7%           8.33MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
init AtA + λI; gra...      1   32.6ms   83.2%  32.6ms   8.00MiB   96.2%  8.00MiB
Diag QUB; normaliz...      1   6.57ms   16.8%  6.57ms    321KiB    3.8%   321KiB
  OLS loop                 1   6.39ms   16.3%  6.39ms    276KiB    3.2%   276KiB
    w = (AᵀA+λI)dₙ₊₁      91   2.03ms    5.2%  22.3μs     0.00B    0.0%    0.00B
    init gradient          1   1.77ms    4.5%  1.77ms     0.00B    0.0%    0.00B
    compute residual       1   1.77ms    4.5%  1.77ms   64.0KiB    0.8%  64.0KiB
    compute search...     9

  0.039551 seconds (1.33 k allocations: 8.420 MiB)


# QREG

In [8]:
h = QBSolvers.default_bandwidth(A)
q = 0.5

0.5

## QUB

In [9]:
x, r, stats = @time solve_QREG(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);
x, r, stats = @time solve_QREG(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            385ms / 100.0%           8.29MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
Diag QLB; normaliz...      1    356ms   92.5%   356ms    292KiB    3.4%   292KiB
  QREG loop                1    356ms   92.4%   356ms    247KiB    2.9%   247KiB
    inner iterations      19    292ms   75.8%  15.3ms   2.94KiB    0.0%     158B
      update g         14.2k    259ms   67.2%  18.3μs     0.00B    0.0%    0.00B
      d = H⁻¹ g        14.2k   14.5ms    3.8%  1.02μs     0.00B    0.0%    0.00B
      x = x + d        14.2k   6.68ms    1.7%   472ns     0.00B    0.0%    0.00B
      check conver...  14.2

  4.609843 seconds (2.13 M allocations: 128.363 MiB, 0.47% gc time, 91.64% compilation time)


────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            384ms / 100.0%           8.30MiB /  99.8%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
Diag QLB; normaliz...      1    364ms   94.7%   364ms    292KiB    3.4%   292KiB
  QREG loop                1    364ms   94.7%   364ms    247KiB    2.9%   247KiB
    inner iterations      19    298ms   77.6%  15.7ms   2.94KiB    0.0%     158B
      update g         14.2k    265ms   69.0%  18.7μs     0.00B    0.0%    0.00B
      d = H⁻¹ g        14.2k   14.4ms    3.8%  1.02μs     0.00B    0.0%    0.00B
      x = x + d        14.2k   6.60ms    1.7%   466ns     0.00B    0.0%    0.00B
      check conver...  14.2

  0.384711 seconds (1.53 k allocations: 8.405 MiB)


## L-BFGS

In [10]:
x, r, stats = @time solve_QREG_lbfgs(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);
x, r, stats = @time solve_QREG_lbfgs(A, b, x_init, p; q=q, h=h, maxiter=10^3, rtol=1e-6, gtol=1e-1, gram=true);

────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            126ms /  99.9%           8.46MiB / 100.0%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
Diag QLB; normaliz...      1    106ms   84.2%   106ms    464KiB    5.4%   464KiB
  QREG loop                1    106ms   84.1%   106ms    420KiB    4.8%   420KiB
    inner iterations      19   43.3ms   34.3%  2.28ms   5.88KiB    0.1%     317B
      w = (AᵀA+λI)...  1.56k   28.6ms   22.6%  18.3μs     0.00B    0.0%    0.00B
      compute sear...  1.56k   9.40ms    7.4%  6.01μs     0.00B    0.0%    0.00B
      update L-BFG...  1.56k   1.21ms    1.0%   776ns     0.00B    0.0%    0.00B
      backtracking...  1.56

  5.871744 seconds (2.86 M allocations: 163.928 MiB, 0.17% gc time, 97.84% compilation time)


────────────────────────────────────────────────────────────────────────────────
                                       Time                    Allocations      
                              ───────────────────────   ────────────────────────
      Tot / % measured:            129ms /  99.9%           8.47MiB /  99.9%    

Section               ncalls     time    %tot     avg     alloc    %tot      avg
────────────────────────────────────────────────────────────────────────────────
Diag QLB; normaliz...      1    108ms   83.7%   108ms    464KiB    5.4%   464KiB
  QREG loop                1    108ms   83.5%   108ms    420KiB    4.8%   420KiB
    inner iterations      19   44.9ms   34.9%  2.36ms   5.88KiB    0.1%     317B
      w = (AᵀA+λI)...  1.56k   29.8ms   23.2%  19.1μs     0.00B    0.0%    0.00B
      compute sear...  1.56k   9.75ms    7.6%  6.24μs     0.00B    0.0%    0.00B
      update L-BFG...  1.56k   1.25ms    1.0%   799ns     0.00B    0.0%    0.00B
      backtracking...  1.56

  0.129323 seconds (1.78 k allocations: 8.590 MiB)
